In [ ]:
%pip install -U py3dti miniaudio soundcard

In [ ]:
from math import sin, cos, radians
from IPython.display import Audio
import numpy as np
from miniaudio import decode_file, SampleFormat
import soundcard
import py3dti

## Setup py3dti

In [ ]:
renderer = py3dti.BinauralRenderer(rate=44100, buffer_size=512, resampled_angular_resolution=5)

### Add listener

In [ ]:
listener = renderer.add_listener(position=None, orientation=None, head_radius=0.0875)
listener.load_hrtf_from_sofa(f'../3dti_AudioToolkit/resources/HRTF/SOFA/3DTI_HRTF_IRC1008_512s_{renderer.rate}Hz.sofa')
# listener.load_hrtf_from_3dti(f'../3dti_AudioToolkit/resources/HRTF/SOFA/3DTI_HRTF_IRC1008_512s_{renderer.rate}Hz.3dti-hrtf') # faster, but less common file format
listener.load_ild_near_field_effect_table(f'../3dti_AudioToolkit/resources/ILD/NearFieldCompensation_ILD_{renderer.rate}.3dti-ild')

In [ ]:
listener.position, listener.orientation

In [ ]:
listener.position = (0, 0, 0) # default listener position
listener.orientation = (1, 0, 0, 0) # default listener orientation

### Add environment

In [ ]:
environment = renderer.add_environment()
environment.load_brir_from_sofa(f'../3dti_AudioToolkit/resources/BRIR/SOFA/3DTI_BRIR_medium_{renderer.rate}Hz.sofa')
# environment.load_brir_from_3dti(f'../3dti_AudioToolkit/resources/BRIR/SOFA/3DTI_BRIR_medium_{renderer.rate}Hz.3dti-brir') # faster, but less common file format

### Add source(s)

In [ ]:
source = renderer.add_source(position=(2, 2, 2), orientation=(2, 1, 1, 1))

In [ ]:
source.position, source.orientation

In [ ]:
renderer, renderer.listener, renderer.sources, renderer.environments

## Read source samples

In [ ]:
file_path = f'../3dti_AudioToolkit/resources/AudioSamples/Anechoic Speech {renderer.rate}.wav'
decoded_file = decode_file(filename=file_path, output_format=SampleFormat.FLOAT32,
                           nchannels=1, sample_rate=renderer.rate)
samples = np.asarray(decoded_file.samples)

## Offline rendering to `np.array`

### Rendering with static listener and sources

In [ ]:
sources = {source: samples}
binaural_samples = renderer.render_offline(sources)

In [ ]:
Audio(binaural_samples.T, rate=renderer.rate, normalize=False)

### Rendering with dynamic listener or sources

In [ ]:
binaural_length = np.max(list(map(len, sources.values())))
binaural_samples = np.zeros((binaural_length, 2), dtype=np.float32)
input_buffer = np.zeros(renderer.buffer_size, dtype=np.float32)

for start in np.arange(0, binaural_length, renderer.buffer_size):
    block_end = min(start + renderer.buffer_size, binaural_length)
    for source, samples in sources.items():
        if start < len(samples):
            # Comment the next line to keep the sources stationary in the location defined above,
            # otherwise it will circle counter-clockwise in the frontal plane
            source.position = (0, sin(radians(start/renderer.buffer_size)), cos(radians(start/renderer.buffer_size)))
            source_end = min(block_end, len(samples))
            source_size = source_end - start
            input_buffer[:source_size] = samples[start:source_end]
            input_buffer[source_size:] = 0
            binaural_samples[start:source_end] += np.column_stack(source.process_anechoic(input_buffer))[:source_size]
    block_size = block_end - start
    for environment in renderer.environments:
        binaural_samples[start:block_end] += np.column_stack(environment.process_virtual_ambisonic_reverb())[:block_size]

In [ ]:
Audio(binaural_samples.T, rate=renderer.rate)

## Real-Time Output

In [ ]:
binaural_length = np.max(list(map(len, sources.values())))
input_buffer = np.zeros(renderer.buffer_size, dtype=np.float32)
output_buffer = np.zeros((renderer.buffer_size, 2), dtype=np.float32)  

with soundcard.default_speaker().player(samplerate=renderer.rate, channels=2) as stereo_speaker:
    for start in np.arange(0, binaural_length, renderer.buffer_size):
        output_buffer.fill(0)
        block_end = min(start + renderer.buffer_size, binaural_length)
        for source, samples in sources.items():
            if start < len(samples):
                # Comment the next line to keep the sources stationary in the location defined above,
                # otherwise it will circle counter-clockwise in the horizontal plane
                source.position = (cos(radians(start/renderer.buffer_size)), sin(radians(start/renderer.buffer_size)), 0)
                source_end = min(block_end, len(samples))
                source_size = source_end - start
                input_buffer[:source_size] = samples[start:source_end]
                input_buffer[source_size:] = 0
                output_buffer += np.column_stack(source.process_anechoic(input_buffer))
        for environment in renderer.environments:
            output_buffer += np.column_stack(environment.process_virtual_ambisonic_reverb())
        block_size = block_end - start
        stereo_speaker.play(output_buffer[:block_size])

## Some more (modifiable) properties with sensible default values

In [ ]:
listener.head_radius, listener.ild_attenuation

In [ ]:
source.spatialization_mode, source.anechoic_processing, source.reverb_processing

In [ ]:
source.far_distance_effect, source.near_field_effect

In [ ]:
source.propagation_delay, source.anechoic_distance_attenuation, source.anechoic_distance_attenuation_smoothing